# Transition states
In this notebook we will locate a transition state by using interpolated force fields from reactant to product(s).

In [ ]:
import veloxchem as vlx
import time
import numpy as np

start_time = time.time()

In [ ]:
charge = 0
multiplicity = 2
basis_set = 'def2-sv(p)'

# rea1 = vlx.Molecule.read_smiles('C1=CCCC(C=O)C1')
# rea1.set_charge(charge)
# rea1.set_multiplicity(multiplicity)
# pro1 = vlx.Molecule.read_smiles('C=CC=C')
# pro1.set_charge(charge)
# pro1.set_multiplicity(multiplicity)
# pro2 = vlx.Molecule.read_smiles('C=CC=O')
# rea1.show()
# pro1.show()
# pro2.show()

# rea1 = vlx.Molecule.read_name('1,4-Dimethyl-1H-1,2,3-triazole')
# rea1.set_charge(charge)
# rea1.set_multiplicity(multiplicity)
# pro1 = vlx.Molecule.read_name('methyl azide')
# pro1.set_charge(charge)
# pro1.set_multiplicity(multiplicity)
# pro2 = vlx.Molecule.read_name('methylacetylene')
# rea1.show()
# pro1.show()
# pro2.show()

# rea1 = vlx.Molecule.read_smiles('C/C(C)=C(C)/C/C(=N\\C#CC)C')
# rea1.set_charge(charge)
# rea1.set_multiplicity(multiplicity)
# pro1 = vlx.Molecule.read_smiles('C=C(C)[C@]/2(C)CC(C)=NC2=C/C')
# pro1.set_charge(charge)
# pro1.set_multiplicity(multiplicity)
# rea1.show()
# pro1.show()

rea1 = vlx.Molecule.read_smiles('CCCC[O]')
rea1.set_charge(charge)
rea1.set_multiplicity(multiplicity)
pro1 = vlx.Molecule.read_smiles('[CH2]CCCO')
pro1.set_charge(charge)
pro1.set_multiplicity(multiplicity)
rea1.show()
pro1.show()

rea_list = [rea1]
pro_list = [pro1]


In [ ]:
ts_guesser = vlx.TransitionStateGuesser()
TS_mol, results = ts_guesser.find_TS(rea_list, pro_list, scf=False)
ts_guesser.show_results()

In [ ]:
TS_mol.set_charge(charge)
TS_mol.set_multiplicity(multiplicity)
basis = vlx.MolecularBasis.read(TS_mol, basis_set)

if multiplicity == 1:
    scf_drv = vlx.ScfRestrictedDriver()
else:
    scf_drv = vlx.ScfUnrestrictedDriver()
scf_drv.xcfun = 'blyp'
scf_drv.conv_thresh = 1e-4
scf_drv.ri_coulomb = True
scf_results = scf_drv.compute(TS_mol, basis)
if multiplicity >= 1:
    xtb_drv = vlx.XtbDriver()
    vibanalysis_drv = vlx.VibrationalAnalysis(xtb_drv)
    vib_results = vibanalysis_drv.compute(TS_mol)
    np.savetxt("hessian.txt", vibanalysis_drv.hessian)

opt_drv = vlx.OptimizationDriver(scf_drv)
if multiplicity >= 1:
    opt_drv.hessian = 'file:hessian.txt'
opt_drv.conv_energy = 1e-04
opt_drv.conv_drms = 1e-02
opt_drv.conv_dmax = 2e-02
opt_drv.conv_grms = 4e-03
opt_drv.conv_gmax = 8e-03
opt_drv.tmax = 0.02
opt_drv.transition = True
opt_results = opt_drv.compute(TS_mol, basis, scf_results)

In [ ]:
opt_drv.show_convergence(opt_results)

In [ ]:
molecule2 = vlx.Molecule.read_xyz_string(opt_results['final_geometry'])
molecule2.set_charge(charge)
molecule2.set_multiplicity(multiplicity)
if multiplicity >= 1:
    xtb_drv = vlx.XtbDriver()
    vibanalysis_drv = vlx.VibrationalAnalysis(xtb_drv)
    vib_results = vibanalysis_drv.compute(TS_mol)
else:
    basis2 = vlx.MolecularBasis.read(molecule2, basis_set)
    scf_drv2 = vlx.ScfRestrictedDriver()
    scf_drv2.xcfun = 'blyp'
    results2 = scf_drv2.compute(molecule2, basis2)
    vibanalysis_drv = vlx.VibrationalAnalysis(scf_drv2)
    vib_results = vibanalysis_drv.compute(molecule2, basis2)

In [ ]:
vibanalysis_drv.animate(vib_results, mode=1)

In [ ]:
print(time.time() - start_time)